<a href="https://colab.research.google.com/github/MIOOR/-/blob/master/%E5%9F%BA%E4%BA%8EPaddlePaddle%E7%9A%84InfoGAN%E7%AE%97%E6%B3%95%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#安装百度飞桨库
!python -m pip install paddlepaddle-gpu==1.7.0.post107 -i https://mirror.baidu.com/pypi/simple

In [ ]:
# 模型的定义以及训练，并在训练过程中展示模型生成的效果
%matplotlib inline
#让matplotlib的输出图像能够直接在notebook上显示
import paddle
import paddle.fluid as fluid
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

#batch 的大小
mb_size = 16
#随机变量长度
Z_dim = 16
#epoch
epoch = 20

#生成随机变量
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def sample_c(m):
    return np.random.multinomial(1, 10*[0.1], size=m)

#生成器模型
#由于aistudio环境限制，使用两层FC网络
def generator(inputs):
    G_h1 = fluid.layers.fc(input = inputs,
                           size = 256,
                           act = "relu",
                           param_attr = fluid.ParamAttr(name="GW1",
                                                       initializer = fluid.initializer.Xavier()),
                           bias_attr = fluid.ParamAttr(name="Gb1",
                                                      initializer = fluid.initializer.Constant()))
    G_prob = fluid.layers.fc(input = G_h1,
                             size = 784,
                             act = "sigmoid",
                             param_attr = fluid.ParamAttr(name="GW2",
                                                       initializer = fluid.initializer.Xavier()),
                             bias_attr = fluid.ParamAttr(name="Gb2",
                                                      initializer = fluid.initializer.Constant()))

    return G_prob

#判别器模型
def discriminator(x):
    D_h1 = fluid.layers.fc(input = x,
                           size = 128,
                           act = "relu",
                           param_attr = fluid.ParamAttr(name="DW1",
                                                       initializer = fluid.initializer.Xavier()),
                           bias_attr = fluid.ParamAttr(name="Db1",
                                                      initializer = fluid.initializer.Constant()))
    D_logit = fluid.layers.fc(input = D_h1,
                            size = 1,
                            act = "sigmoid",
                            param_attr = fluid.ParamAttr(name="DW2",
                                                       initializer = fluid.initializer.Xavier()),
                            bias_attr = fluid.ParamAttr(name="Db2",
                                                      initializer = fluid.initializer.Constant()))

    return D_logit

#c的概率近似（对于输入X）
def Q(x):
    Q_h1 = fluid.layers.fc(input = x,
                           size = 128,
                           act = "relu",
                           param_attr = fluid.ParamAttr(name="QW1",
                                                       initializer = fluid.initializer.Xavier()),
                           bias_attr = fluid.ParamAttr(name="Qb1",
                                                      initializer = fluid.initializer.Constant()))
    Q_prob = fluid.layers.fc(input = Q_h1,
                             size = 10,
                             act = "softmax",
                             param_attr = fluid.ParamAttr(name="QW2",
                                                       initializer = fluid.initializer.Xavier()),
                             bias_attr = fluid.ParamAttr(name="Qb2",
                                                      initializer = fluid.initializer.Constant()))

    return Q_prob

#G优化程序
G_program = fluid.Program()
with fluid.program_guard(G_program, fluid.default_startup_program()):
    Z = fluid.layers.data(name='Z', shape=[Z_dim], dtype='float32')
    c = fluid.layers.data(name='c', shape=[10], dtype='float32')
    #合并输入
    inputs = fluid.layers.concat(input=[Z, c], axis = 1)
    G_sample = generator(inputs)
    D_fake = discriminator(G_sample)  
    G_loss = 0.0 - fluid.layers.reduce_mean(fluid.layers.log(D_fake + 1e-8))
    theta_G = ["GW1", "Gb1", "GW2", "Gb2"]
    G_optimizer = fluid.optimizer.AdamOptimizer()
    G_optimizer.minimize(G_loss, parameter_list=theta_G)
    

#D优化程序
D_program = fluid.Program()
with fluid.program_guard(D_program, fluid.default_startup_program()):
    Z = fluid.layers.data(name='Z', shape=[Z_dim], dtype='float32')
    c = fluid.layers.data(name='c', shape=[10], dtype='float32')
    X = fluid.layers.data(name='X', shape=[784], dtype='float32')
    X = X * 0.5 + 0.5

    inputs = fluid.layers.concat(input=[Z, c], axis = 1)
    G_sample = generator(inputs)
    D_real = discriminator(X)
    D_fake = discriminator(G_sample)  

    D_loss = 0.0 - fluid.layers.reduce_mean(fluid.layers.log(D_real + 1e-8) 
                                            + fluid.layers.log(1.0 - D_fake + 1e-8))
    theta_D = ["DW1", "Db1", "DW2", "Db2"]
    D_optimizer = fluid.optimizer.AdamOptimizer()
    D_optimizer.minimize(D_loss, parameter_list=theta_D)

#Q优化程序
Q_program = fluid.Program()
with fluid.program_guard(Q_program, fluid.default_startup_program()):   
    Z = fluid.layers.data(name='Z', shape=[Z_dim], dtype='float32')
    c = fluid.layers.data(name='c', shape=[10], dtype='float32')

    inputs = fluid.layers.concat(input=[Z, c], axis = 1)
    G_sample = generator(inputs)
    Q_c_given_x = Q(G_sample)
    #最小化熵
    Q_loss = fluid.layers.reduce_mean(
        0.0 - fluid.layers.reduce_sum(
            fluid.layers.elementwise_mul(fluid.layers.log(Q_c_given_x + 1e-8), c), 1))   
    theta_Q = ["GW1", "Gb1", "GW2", "Gb2", 
               "QW1", "Qb1", "QW2", "Qb2"]
    Q_optimizer = fluid.optimizer.AdamOptimizer()
    Q_optimizer.minimize(Q_loss, parameter_list = theta_Q)

#Inference
Infer_program = fluid.Program()
with fluid.program_guard(Infer_program, fluid.default_startup_program()):   
    Z = fluid.layers.data(name='Z', shape=[Z_dim], dtype='float32')
    c = fluid.layers.data(name='c', shape=[10], dtype='float32')
    inputs = fluid.layers.concat(input=[Z, c], axis = 1)
    G_sample = generator(inputs)

#读入数据，只载入训练集
paddle.dataset.common.DATA_HOME = './data65'   #更改下载路径，使用提前保存好的MNIST数据集
train_reader = paddle.batch(
                paddle.reader.shuffle(
                paddle.dataset.mnist.train(), buf_size=500),    #paddle.dataset.mnist.train()
                batch_size=mb_size)

#Executor
# exe = fluid.Executor(fluid.CPUPlace())  #CUDAPlace(0)
# exe.run(program=fluid.default_startup_program())
# 定义使用CPU还是GPU，使用CPU时use_cuda = False,使用GPU时use_cuda = True
use_cuda = True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
# 获取测试程序
test_program = fluid.default_main_program().clone(for_test=True)
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())


it = 0
for _ in range(epoch):
    for data in train_reader():
        it += 1
        #获取训练集图像
        X_mb = [data[i][0] for i in range(mb_size)]
        #生成噪声
        Z_noise = sample_Z(mb_size, Z_dim)
        c_noise = sample_c(mb_size)

        feeding_withx= {"X" : np.array(X_mb).astype('float32'), 
                        "Z" : np.array(Z_noise).astype('float32'), 
                        "c" : np.array(c_noise).astype('float32')}
        
        feeding = {"Z" : np.array(Z_noise).astype('float32'), 
                   "c" : np.array(c_noise).astype('float32')}
        #三层优化
        D_loss_curr = exe.run(feed = feeding_withx, program = D_program, fetch_list = [D_loss])

        G_loss_curr = exe.run(feed = feeding, program = G_program, fetch_list = [G_loss])

        Q_loss_curr = exe.run(feed = feeding, program = Q_program, fetch_list = [Q_loss])
        if it % 1000 == 0:
            print(str(it) + ' | ' 
                  + str (D_loss_curr[0][0]) + ' | ' 
                  + str (G_loss_curr[0][0]) + ' | ' 
                  + str (Q_loss_curr[0][0]))
        if it % 10000 == 0:
            #显示模型生成结果
            Z_noise_ = sample_Z(mb_size, Z_dim)
            idx1 = np.random.randint(0, 10)
            idx2 = np.random.randint(0, 10)
            # idx3 = np.random.randint(0, 10)
            # idx4 = np.random.randint(0, 10)
            c_noise_ = np.zeros([mb_size, 10])
            c_noise_[range(8), idx1] = 1.0
            c_noise_[range(8, 16), idx2] = 1.0
            # c_noise_[range(16, 24), idx3] = 1.0
            # c_noise_[range(24, 32), idx4] = 1.0
            feeding_ = {"Z" : np.array(Z_noise_).astype('float32'), 
                       "c" : np.array(c_noise_).astype('float32')}
            samples = exe.run(feed = feeding_,
                              program = Infer_program,
                              fetch_list = [G_sample])
            # 保存固化后用于infer的模型，方便后续使用
            fluid.io.save_inference_model(dirname='freeze_model', executor=exe, feeded_var_names=['Z', 'c'], target_vars=[G_sample],main_program=Infer_program)
            for i in range(mb_size):
                ax = plt.subplot(4, 8, 1 + i)
                plt.axis('off')
                ax.set_xticklabels([])
                ax.set_yticklabels([])
                ax.set_aspect('equal')
                plt.imshow(np.reshape(samples[0][i], [28,28]), cmap='Greys_r')
            plt.show()

In [ ]:
# 使用保存的模型进行随机生成
exe = fluid.Executor(fluid.CPUPlace())
[infer_program, feed_list, fetch_list] = fluid.io.load_inference_model('freeze_model', exe)
print(feed_list)
Z_noise_ = sample_Z(mb_size, Z_dim)
idx1 = np.random.randint(0, 10)
idx2 = np.random.randint(0, 10)
idx3 = np.random.randint(0, 10)
idx4 = np.random.randint(0, 10)
c_noise_ = np.zeros([mb_size, 10])
c_noise_[range(8), idx1] = 1.0
c_noise_[range(8, 16), idx2] = 1.0
# c_noise_[range(16, 24), idx3] = 1.0
# c_noise_[range(24, 32), idx4] = 1.0
feeding_ = {feed_list[0] : np.array(Z_noise_).astype('float32'), 
           feed_list[1] : np.array(c_noise_).astype('float32')}
samples = exe.run(feed = feeding_,
                  program = Infer_program,
                  fetch_list = fetch_list)
for i in range(mb_size):
    ax = plt.subplot(4, 8, 1 + i)
    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_aspect('equal')
    plt.imshow(np.reshape(samples[0][i], [28,28]), cmap='Greys_r')
plt.show()